In [ ]:
!nvidia-smi

Sun Feb 12 18:00:11 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    26W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
 !rm -rf ./xlm-roberta-ner-hin

In [ ]:
!pip install numpy seqeval pandas matplotlib scikit-learn transformers datasets

In [ ]:
import torch

if torch.cuda.is_available():
    print("GPU is enabled.")
    print("device count: {}, current device: {}".format(torch.cuda.device_count(), torch.cuda.current_device()))
else:
    print("GPU is not enabled.")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

GPU is enabled.
device count: 1, current device: 0


In [ ]:
from datasets import load_dataset
train_dataset = load_dataset('ai4bharat/naamapadam','kn',split='train[:9%]')

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset naamapadam downloaded and prepared to /root/.cache/huggingface/datasets/ai4bharat___naamapadam/kn/1.0.0/c1b045180d60b208d2468bdad897d04461f08c7137c04a85220697b1bef7df9a. Subsequent calls will reuse this data.


In [ ]:
train_dataset

Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 42459
})

In [ ]:
val_dataset = load_dataset('ai4bharat/naamapadam','kn',split='validation[:5%]')

In [ ]:
val_dataset

Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 119
})

In [ ]:
tags = train_dataset.features["ner_tags"].feature
index2tag = {idx: tag for idx, tag in enumerate(tags.names)}
tag2index = {tag: idx for idx, tag in enumerate(tags.names)}
tag2index

{'O': 0,
 'B-PER': 1,
 'I-PER': 2,
 'B-ORG': 3,
 'I-ORG': 4,
 'B-LOC': 5,
 'I-LOC': 6}

In [ ]:
num_labels = len(tag2index)

In [ ]:
from transformers import AutoTokenizer
xlmr_tokenizer = AutoTokenizer.from_pretrained("cfilt/HiNER-original-xlm-roberta-large")
print("The maximum length for the inputs is {}".format(xlmr_tokenizer.model_max_length))

The maximum length for the inputs is 512


In [ ]:
for ind, val in enumerate(train_dataset["tokens"][2]):
  print(val,train_dataset['ner_tags'][2][ind])

ಆಕೆಯ 0
ತಂದೆ 0
ತನ್ನ 0
ಮಕ್ಕಳಿಲ್ಲದ 0
ಸೋದರಸಂಬಂಧಿ 0
ಕುಂತಿಭೋಜನಿಗೆ 1
ಕುಂತಿಯನ್ನು 1
ಕೊಟ್ಟರು 0
. 0


In [ ]:
import pandas as pd
test_encoding = xlmr_tokenizer("".join(train_dataset["tokens"][2]))
pd.DataFrame(
    [xlmr_tokenizer.convert_ids_to_tokens(test_encoding.input_ids)],
    index=["tokens"])

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
tokens,<s>,▁जाने,-,मा,ने,वै,ज्ञान,िक,सि,वान,...,नि,युक्त,क,िया,ग,या,ह,ै,।,</s>


In [ ]:
# Tokenize all texts and align the labels with them.
padding = "max_length"
text_column_name = 'tokens'
label_column_name = 'ner_tags'
def tokenize_and_align_labels(examples):
    tokenized_inputs = xlmr_tokenizer(
        examples[text_column_name],
        padding=padding,
        truncation=True,
        max_length=512,
        # We use this argument because the texts in our dataset are lists of words (with a label for each word).
        is_split_into_words=True,
    )
    labels = []
    for i, label in enumerate(examples[label_column_name]):
        # print('=====')
        # print('{} {}'.format(i,label)) #ak
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx

        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
tokenized_ds = train_dataset.map(
    tokenize_and_align_labels,
    remove_columns=["tokens", "ner_tags"],
    batched=True,
    batch_size=128)

  0%|          | 0/332 [00:00<?, ?ba/s]

In [ ]:
len(tokenized_ds)

42459

In [ ]:
val_tokenized = val_dataset.map(
    tokenize_and_align_labels,
    remove_columns=["tokens", "ner_tags"],
    batched=True,
    batch_size=128)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
# pd.DataFrame(
#     [xlmr_tokenizer.convert_ids_to_tokens(tokenized_ds[0]["input_ids"]), tokenized_ds[0]["labels"]],
#     index=["tokens", "ner_tags"])

for ind, val in enumerate(xlmr_tokenizer.convert_ids_to_tokens(tokenized_ds[0]["input_ids"])):
  print(val,tokenized_ds[0]["labels"][ind])

<s> -100
▁से 0
क्टर -100
▁55 0
/ -100
56 -100
▁के 0
▁एस 0
एच -100
ओ -100
▁अरविंद 1
▁कुमार 2
▁ने 0
▁बताया 0
▁कि 0
▁इस 0
▁मामले 0
▁में 0
▁आई 0
पी -100
सी -100
▁की 0
▁धारा 0
▁3 0
76 -100
▁- 0
▁डी 0
▁( 0
▁ग 0
ै -100
ंग -100
रे -100
प -100
▁) 0
▁के 0
▁तहत 0
▁मामला 0
▁दर्ज 0
▁कर 0
▁लिया 0
▁गया 0
▁है 0
▁। 0
</s> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> 

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir = "xlm-roberta-ner-kan",
    log_level = "error",
    num_train_epochs = 1,
    per_device_train_batch_size = 12,
    per_device_eval_batch_size = 12,
    evaluation_strategy = "steps",
    eval_steps = 500, # Evaluation and Save happens every 10 steps
    fp16 = True,
    logging_steps = len(tokenized_ds),
    save_total_limit=5,
    load_best_model_at_end= True,
    push_to_hub = False
    )

In [ ]:
%env TOKENIZERS_PARALLELISM=false

env: TOKENIZERS_PARALLELISM=false


In [ ]:
import numpy as np
# see https://huggingface.co/spaces/evaluate-metric/seqeval
from seqeval.metrics import f1_score

def metrics_func(eval_arg):
    preds = np.argmax(eval_arg.predictions, axis=2)
    batch_size, seq_len = preds.shape
    y_true, y_pred = [], []
    for b in range(batch_size):
        true_label, pred_label = [], []
        for s in range(seq_len):
            if eval_arg.label_ids[b, s] != -100:  # -100 must be ignored
                true_label.append(index2tag[eval_arg.label_ids[b][s]])
                pred_label.append(index2tag[preds[b][s]])
        y_true.append(true_label)
        y_pred.append(pred_label)
    return {"f1": f1_score(y_true, y_pred)}

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(
    xlmr_tokenizer,
    return_tensors="pt")

In [ ]:
from transformers import AutoConfig, Trainer
from transformers.models.roberta.modeling_roberta import RobertaForTokenClassification

xlmr_config = AutoConfig.from_pretrained(
    "xlm-roberta-base",
    num_labels=tags.num_classes,
    id2label=index2tag,
    label2id=tag2index
)
model = (RobertaForTokenClassification
         .from_pretrained("xlm-roberta-base", config=xlmr_config)
         .to(device))

trainer = Trainer(
    model = model,
    args = training_args,
    data_collator = data_collator,
    compute_metrics = metrics_func,
    train_dataset = tokenized_ds,
    eval_dataset = val_tokenized,
    tokenizer = xlmr_tokenizer
)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing RobertaForTokenClassification: ['lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,F1
500,No log,0.312530,0.675676
1000,No log,0.270149,0.716724
1500,No log,0.252587,0.750853
2000,No log,0.260286,0.756944
2500,No log,0.234753,0.771331
3000,No log,0.240310,0.741259
3500,No log,0.237141,0.754325


TrainOutput(global_step=3539, training_loss=0.3138058660528221, metrics={'train_runtime': 2004.5918, 'train_samples_per_second': 21.181, 'train_steps_per_second': 1.765, 'total_flos': 1.1094900513168384e+16, 'train_loss': 0.3138058660528221, 'epoch': 1.0})

In [ ]:
import os

os.makedirs("./trained_ner_classifier_kan", exist_ok=True)
#trainer.save_model("./trained_ner_classifier_jp")
if hasattr(trainer.model, "module"):
    trainer.model.module.save_pretrained("./trained_ner_classifier_kan")
else:
    trainer.model.save_pretrained("./trained_ner_classifier_kan")

In [ ]:
!zip -r /content/trained_ner_classifier_kan.zip /content/trained_ner_classifier_kan

  adding: content/trained_ner_classifier_kan/ (stored 0%)
  adding: content/trained_ner_classifier_kan/config.json (deflated 53%)
  adding: content/trained_ner_classifier_kan/pytorch_model.bin (deflated 31%)


In [ ]:
import torch
from transformers import AutoConfig

xlmr_config = AutoConfig.from_pretrained(
    "xlm-roberta-base",
    num_labels=tags.num_classes,
    id2label=index2tag,
    label2id=tag2index
)
model = (RobertaForTokenClassification
         .from_pretrained("./trained_ner_classifier_kan", config=xlmr_config)
         .to(device))

In [ ]:
from datasets import Dataset
import torch
from torch.utils.data import DataLoader
import pandas as pd

sample_encoding = xlmr_tokenizer([
   "ನಗರದ ಯಲಹಂಕ ಏರ್​ ಬೇಸ್​ನಲ್ಲಿ ನಡೆಯುವ ಏರೋ ಇಂಡಿಯಾ ಏರ್​​ ಶೋ ಉದ್ಘಾಟನೆ ಸಂಬಂಧ ಪ್ರಧಾನಿ ನರೇಂದ್ರ ಮೋದಿ ಬೆಂಗಳೂರಿಗೆ ಬಂದಿಳಿದಿದ್ದಾರೆ",
   "ಯಶರಾಜ್ ನಾಳೆ ಗೋರಖ್‌ಪುರಕ್ಕೆ ಹೋಗುತ್ತಿದ್ದಾರೆ, ಅದು ಭಾರತದ ನಗರ"
  # "सेक्टर 55/56 क एसएचओ अरविंद कुमार ने बताया कि इस मामले"
], truncation=True, max_length=512)
sample_dataset = Dataset.from_dict(sample_encoding)
sample_dataset = sample_dataset.with_format("torch")

sample_dataloader = DataLoader(sample_dataset, batch_size=1)
tokens = []
labels = []
for batch in sample_dataloader:
    # predict
    with torch.no_grad():
        output = model(batch["input_ids"].to(device), batch["attention_mask"].to(device))
    predicted_label_id = torch.argmax(output.logits, axis=-1).cpu().numpy()
    # create output
    tokens.append(xlmr_tokenizer.convert_ids_to_tokens(batch["input_ids"][0]))
    labels.append([index2tag[i] for i in predicted_label_id[0]])

pd.DataFrame([tokens[1], labels[1]], index=["Tokens", "Tags"])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
Tokens,<s>,▁,ಯ,ಶ,ರಾಜ್,▁ನಾಳೆ,▁ಗೋ,ರ,ಖ್,▁,ಪುರ,ಕ್ಕೆ,▁ಹೋಗ,ುತ್ತಿದ್ದಾರೆ,",",▁ಅದು,▁ಭಾರತದ,▁ನಗರ,</s>
Tags,O,B-PER,B-PER,B-PER,I-PER,O,B-LOC,B-LOC,B-LOC,I-LOC,I-LOC,O,O,O,O,O,B-LOC,O,O


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [ ]:
!cp /content/trained_ner_classifier_kan.zip '/content/gdrive/MyDrive/cflit-hiner-xlm-roberta-weights/'
!ls -lt '/content/gdrive/MyDrive/cflit-hiner-xlm-roberta-weights/' 

total 1501148
-rw------- 1 root root 763895852 Feb 12 18:50 trained_ner_classifier_kan.zip
drwx------ 2 root root      4096 Feb 12 16:42 trained-ner-xlm-hindi-ai4bharat
-rw------- 1 root root 773274459 Feb 12 15:13 trained_ner_classifier_hin.zip
